In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.display.max_columns = None

import pickle

In [2]:
t_seps = [10, 12, 14, 16, 18, 20, 22, 24]

# trajs = range(2100, 2150, 10)
trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results/')]) 
print(trajs)

def mean_err(arr):  # shape: (trajs, ...)
    mean = arr.mean(axis=0)
    err = arr.std(axis=0) / np.sqrt(arr.shape[0])  
    return mean, err  

[2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100, 2110, 2120, 2130, 2140, 2150, 2160, 2170, 2180, 2190, 2200, 2210, 2220, 2230, 2240, 2250]


In [3]:
# traj = 2000
# data = pickle.load(open(f'./Kaon_results/{traj}.pkl', 'rb'))
# data.keys()

In [16]:
# # Without disconnected diagrams
# coefficients = {
#     'cs_sBar_d_T1D1': -1/np.sqrt(6), 'cs_sBar_d_T1D2': 2/np.sqrt(6), 
# #     'cs_sBar_d_T2D1': 2/np.sqrt(6), 'cs_sBar_d_T2D2': -2/np.sqrt(6),   # disconnected
    
#     'cs_Hw_T1D1Q1': 1/np.sqrt(6), 'cs_Hw_T1D1Q2': -1/np.sqrt(6), 
#     'cs_Hw_T2D1Q1': 1/np.sqrt(6), 'cs_Hw_T2D1Q2': -1/np.sqrt(6), 
#     'cs_Hw_T2D2Q1': -2/np.sqrt(6), 'cs_Hw_T2D2Q2': 2/np.sqrt(6), 
# #     'cs_Hw_T3D1Q1': -2/np.sqrt(6), 'cs_Hw_T3D1Q2': 2/np.sqrt(6),    # disconnected
# #     'cs_Hw_T3D2Q1': 2/np.sqrt(6), 'cs_Hw_T3D2Q2': -2/np.sqrt(6)   # disconnected
# }

# With disconnected diagrams
coefficients = {
    'cs_sBar_d_T1D1': -1/np.sqrt(6), 'cs_sBar_d_T1D2': 2/np.sqrt(6), 
    
    'cs_sBar_d_T2D1': 2/np.sqrt(6), 'cs_sBar_d_T2D2': -2/np.sqrt(6),   # disconnected
    
    'cs_Hw_T1D1Q1': 1/np.sqrt(6), 'cs_Hw_T1D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D1Q1': 1/np.sqrt(6), 'cs_Hw_T2D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D2Q1': -2/np.sqrt(6), 'cs_Hw_T2D2Q2': 2/np.sqrt(6), 
    
    'cs_Hw_T3D1Q1': -2/np.sqrt(6), 'cs_Hw_T3D1Q2': 2/np.sqrt(6),    # disconnected
    'cs_Hw_T3D2Q1': 2/np.sqrt(6), 'cs_Hw_T3D2Q2': -2/np.sqrt(6)   # disconnected
}


diagrams = list(coefficients.keys())

Q1_diagrams = [d for d in diagrams if d.endswith('Q1')] 
Q2_diagrams = [d for d in diagrams if d.endswith('Q2')] 
sBar_d_diagrams = [d for d in diagrams if d.startswith('cs_sBar_d')]

print(f'Q1_diagrams: {Q1_diagrams}')
print(f'Q2_diagrams: {Q2_diagrams}')
print(f'sBar_d_diagrams: {sBar_d_diagrams}')

Q1_diagrams: ['cs_Hw_T1D1Q1', 'cs_Hw_T2D1Q1', 'cs_Hw_T2D2Q1', 'cs_Hw_T3D1Q1', 'cs_Hw_T3D2Q1']
Q2_diagrams: ['cs_Hw_T1D1Q2', 'cs_Hw_T2D1Q2', 'cs_Hw_T2D2Q2', 'cs_Hw_T3D1Q2', 'cs_Hw_T3D2Q2']
sBar_d_diagrams: ['cs_sBar_d_T1D1', 'cs_sBar_d_T1D2', 'cs_sBar_d_T2D1', 'cs_sBar_d_T2D2']


In [17]:
def calc_cs(traj):
    data = pickle.load(open(f'./Kaon_results/{traj}.pkl', 'rb'))
    
    eta_Q1_K = sum([data[d] * coefficients[d] for d in Q1_diagrams])
    eta_Q2_K = sum([data[d] * coefficients[d] for d in Q2_diagrams])
    eta_sBar_d_K = sum([data[d] * coefficients[d] for d in sBar_d_diagrams])
    
    eta_Q1_K = eta_Q1_K.real # ??? should I take real part; seems like imag part is small
    eta_Q2_K = eta_Q2_K.real 
    eta_sBar_d_K = eta_sBar_d_K.real

    c_s_Q1 = -  eta_Q1_K / eta_sBar_d_K
    c_s_Q2 = -  eta_Q2_K / eta_sBar_d_K
    
    return c_s_Q1, c_s_Q2   # shape (tsep, tx)

In [18]:
c_s_Q1, c_s_Q2 = [], []
for traj in trajs:
    c_s_Q1_one_traj, c_s_Q2_one_traj = calc_cs(traj)   # shape (tsep, tx)
    c_s_Q1.append(c_s_Q1_one_traj)
    c_s_Q2.append(c_s_Q2_one_traj)
c_s_Q1 = np.array(c_s_Q1)  # (trajs, tsep, tx)
c_s_Q2 = np.array(c_s_Q2)

c_s_Q1_avg, c_s_Q1_err = mean_err(c_s_Q1)
c_s_Q2_avg, c_s_Q2_err = mean_err(c_s_Q2)

In [19]:
# zyd: spikes are probably because that denominator is small at these positions
# Result is messy; but result is very good if we comment out the disconnected diagrams

fig = go.Figure()
for i, t_sep in enumerate(t_seps[:5]):
    fig.add_trace(
        go.Scatter(x=list(range(t_sep)), y=c_s_Q1_avg[i][:t_sep], name=f'tsep={t_sep}',
                  error_y=dict(type='data', array=c_s_Q1_err[i][:t_sep]))
    )
fig.update_layout(title_text='c_s for Q1', width=600, height=400)
fig.show()

fig = go.Figure()
for i, t_sep in enumerate(t_seps[:5]):
    fig.add_trace(
        go.Scatter(x=list(range(t_sep)), y=c_s_Q2_avg[i][:t_sep], name=f'tsep={t_sep}',
                  error_y=dict(type='data', array=c_s_Q2_err[i][:t_sep]))
    )
fig.update_layout(title_text='c_s for Q2', width=600, height=400)
fig.show()

In [8]:
# c_s_Q1, c_s_Q2 = calc_cs(2100)

# fig = go.Figure()
# for i, t_sep in enumerate(t_seps[:5]):
#     fig.add_trace(go.Scatter(x=list(range(t_sep)), y=c_s_Q1[i][:t_sep], name=f'tsep={t_sep}'))
# fig.update_layout(title_text='c_s for Q1', width=600, height=400)
# fig.show()


# fig = go.Figure()
# for i, t_sep in enumerate(t_seps[:5]):
#     fig.add_trace(go.Scatter(x=list(range(t_sep)), y=c_s_Q2[i][:t_sep], name=f'tsep={t_sep}'))
# fig.update_layout(title_text='c_s for Q2', width=600, height=400)
# fig.show()